In [18]:
import matplotlib.pyplot as plt
import os
import sys
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Dropout, Embedding, SimpleRNN, LSTM, Bidirectional
from keras.callbacks import History

The Embedding layer is defined as the first hidden layer of a network. It must specify 3 arguments:

It must specify 3 arguments:

* **input_dim**: This is the size of the vocabulary in the text data. For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.
* **output_dim**: This is the size of the vector space in which words will be embedded. It defines the size of the output vectors from this layer for each word. For example, it could be 32 or 100 or even larger. Test different values for your problem.
* **input_length**: This is the length of input sequences, as you would define for any input layer of a Keras model. For example, if all of your input documents are comprised of 1000 words, this would be 1000.

# Zad.

Poberz dane z konkursu http://2017.poleval.pl/index.php/tasks/ Task 2. Sentiment analysis

Pojedyńczy plik można pobrać z tąd:
https://www.dropbox.com/sh/tjq47ybybgnrbel/AAAVbp0UkQTAbKWVMIi5mtHpa?dl=0

## Zad
Zbuduj model na podstawie warstwy Embedding, która się uczy.


In [4]:
file_with_filtered_embeddings = "Dane/data_poleval/embeddings.txt"

words2ids = {}
embeddings = []

embeddings.append(np.zeros(300)) # rezerwujemy embeddingi na paddin i nieznane slowa
embeddings.append(np.zeros(300))

i = 0
with open(file_with_filtered_embeddings,"r", encoding="utf8")as f:
    for line in f:
        toks = line.split(" ")
        word = toks[0]
        embeddings.append(np.array([float(x) for x in toks[1:]]))
        words2ids[word] = i+2 # +3 - przesuniecie po to zeby specjalne embeddingi byly na pozycji 0 i 1
        i = i + 1


embeddings = np.array(embeddings)
print(embeddings.shape)

(5000, 300)


In [7]:
words2ids["Słodkawy"]

2731

In [8]:
embeddings[words2ids["Słodkawy"]]

array([ 0.0695675 , -0.0123177 ,  0.0190564 ,  0.0205853 , -0.0586069 ,
        0.123962  , -0.0113998 ,  0.0486168 , -0.0593128 ,  0.0244056 ,
        0.00806909, -0.0123139 ,  0.0318254 ,  0.0375928 ,  0.0488764 ,
       -0.0311151 , -0.0571845 ,  0.00886492, -0.023981  , -0.0729585 ,
        0.00780728, -0.0103228 ,  0.0451543 , -0.00375595, -0.01016   ,
        0.0157066 ,  0.0933215 ,  0.00742586,  0.0662036 ,  0.0147866 ,
       -0.0326982 , -0.105277  ,  0.0939024 , -0.141137  , -0.0390246 ,
        0.0307554 , -0.124878  ,  0.092467  , -0.0449775 ,  0.0278756 ,
        0.0120974 ,  0.0402319 , -0.0602183 ,  0.117348  , -0.0286395 ,
        0.0226578 , -0.0100841 ,  0.0928551 ,  0.0676152 , -0.0280563 ,
        0.0516412 ,  0.0899037 , -0.0634045 ,  0.0149937 , -0.0472655 ,
       -0.0638853 , -0.00977132, -0.0440674 ,  0.00945409, -0.0873957 ,
        0.0113474 ,  0.00536351,  0.0406797 ,  0.051127  ,  0.0194782 ,
        0.00793247,  0.00938793,  0.0813627 , -0.0323478 ,  0.00

In [5]:
from keras.preprocessing import sequence as seq
def load_and_transform_data_to_phrases(labels, parents, tokens, words2ids):

    """
    Dokumentacja
    """
    

    transform_label = {'-1':0, '0':1, '1':2}
    
    l = open(labels, "r", encoding="utf8")
    labels = [[transform_label[y] for y in x.split()] for x in l.readlines()] 
    l.close()

    p = open(parents,"r", encoding="utf8")
    parents = [[int(y) for y in x.split()] for x in p.readlines()]
    p.close()

    t = open(tokens,"r", encoding="utf8")
    tokens = [x.split() for x in t.readlines()]
    t.close()
    
    k = 0
    result = []
    
    for labels_i,parents_i,tokens_i in zip(labels,parents,tokens):
        
        k = k + 1
         
        s = []
        for i in range(len(tokens_i)):
            s.append([i,int(parents_i[i]),labels_i[i],tokens_i[i]])


        if len(s) == 1: #przypadek gdy fraza sklada sie z jednego tokena

            result.append((\
                                  tokens[0],
                                  np.array([words2ids.get(tokens[0], 1)]),\
                                  np.array(labels_i[0]) \
                              ))    
                           
        else: 
            
            for i in range(len(s)): 
                children = []
                for j in range(len(s)):
                    if s[j][1] == i+1:
                        children.append(s[j][0])
                s[i].append(children)

                
            words = [x[0] for x in s]
            children = [x[4] for x in s]
            tokens = [x[3] for x in s]
            labels_in_batch = [x[2] for x in s]
        
            phrases = [[k] for k in range(len(children))]
            for i in range(len(children)):
                for e in phrases[i]:
                    phrases[i].extend(children[e])
           
            phrases = [ np.sort(x) for x in phrases]
          
            phrases = list(zip([np.array(tokens_i)[x] for x in phrases],
                               [np.array([words2ids.get(t,1) for t in tokens_i])[x] for x in phrases],
                               labels_i))

            result.extend(phrases)
           
    return result


In [6]:
train_data = load_and_transform_data_to_phrases("Dane/data_poleval/training-treebank/rev_labels.txt", "Dane/data_poleval/training-treebank/rev_parents.txt","Dane/data_poleval/training-treebank/rev_sentence.txt",words2ids)
test_data = load_and_transform_data_to_phrases("Dane/data_poleval/poleval_test/gold_labels", "Dane/data_poleval/poleval_test/polevaltest_parents.txt","Dane/data_poleval/poleval_test/polevaltest_sentence.txt",words2ids) 

In [1]:

print('Accuracy: %f' % (accuracy))

In [2]:
plt.plot(history_1.history['categorical_accuracy'], label = "tarina")
plt.plot(history_1.history['val_categorical_accuracy'], label = "test")


plt.legend()
plt.show()

# Zad
Użyj Pretrain embedding


In [3]:
print('Accuracy: %f' % (accuracy))

In [4]:
plt.plot(history_1.history['categorical_accuracy'], label = "tarina Trainable")
plt.plot(history_1.history['val_categorical_accuracy'], label = "test Trainable")

plt.plot(history_2.history['categorical_accuracy'], label = "tarina Not Trainable")
plt.plot(history_2.history['val_categorical_accuracy'], label = "test Not Trainable")


plt.legend()
plt.show()

# Zad
Użyj architektury
```python
model = Sequential()
model.add(Embedding(5000, 300,
                   input_length=max_len,
                  weights=[embeddings],
                  trainable=False))
model.add(LSTM(10))
model.add(Dense(3, activation='softmax'))
model.summary()
```

# Zad
Użyj architektury
```python
model = Sequential()
model.add(Embedding(5000, 300,
                   input_length=max_len,
                  weights=[embeddings],
                  trainable=False))
model.add(Conv1D(50,3))
model.add(LSTM(10))
model.add(Dense(3, activation='softmax'))
model.summary()
```

Co dokładnie robi 

```python
from keras.layers.convolutional import Conv1D
```

# Zad
Użyj architektury

```python
model = Sequential()
model.add(Embedding(5000, 300,
                   input_length=max_len,
                  weights=[embeddings],
                  trainable=False))
model.add(TimeDistributed(Dense(10,activation="tanh")))
model.add(LSTM(10))
model.add(Dense(3, activation='softmax'))
model.summary()
```

Co dokładnie robi 

```python
from keras.layers import TimeDistributed
```
